In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import cv2
import matplotlib.pyplot as plt
#%matplotlib inline
import imutils
import shutil
import pickle

from keras.utils import image_dataset_from_directory
from keras.models import Model,load_model
from keras.layers import Conv2D,Input,ZeroPadding2D,BatchNormalization,Flatten,Activation,Dense,MaxPooling2D

2023-07-29 21:50:41.765353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
image_dir = "archive-2/Training"

In [3]:
def crop_brain_contour(image, plot=False):
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    # extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]

    if plot:
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.imshow(image)
        plt.tick_params(axis='both', which='both', top=False, bottom=False, left=False, right=False, labelbottom=False,
                        labeltop=False, labelleft=False, labelright=False)
        plt.title('Original Image')
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)
        plt.tick_params(axis='both', which='both', top=False, bottom=False, left=False, right=False, labelbottom=False,
                        labeltop=False, labelleft=False, labelright=False)
        plt.title('Cropped Image')
        plt.show()

    return new_image

In [10]:
destination_path = "new_trainingdata"
if not os.path.exists(destination_path):
    os.makedirs(destination_path)
for type_folder in os.listdir(image_dir):
    folder_path= os.path.join(image_dir,type_folder)
    # print(folder_path)
    destination_folder=os.path.join(destination_path,type_folder)
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    #if type_folder != ".DS_Store":
    for img_name in os.listdir(folder_path):
        if type_folder == ".DS_Store":
            os.rmdir(type_folder)
        else:
            ex_img = cv2.imread(os.path.join(folder_path,img_name))
            ex_crop_img = crop_brain_contour(ex_img, False)
            cv2.imwrite(os.path.join(destination_folder,img_name),ex_crop_img)

In [11]:
train_dt, val_dt = image_dataset_from_directory(
    directory = destination_path,
    labels="inferred",
    batch_size = 32,
    image_size = (224, 224),
    subset = "both",
    validation_split = 0.2,
    seed = 42)

Found 5712 files belonging to 4 classes.
Using 4570 files for training.
Using 1142 files for validation.


In [12]:
class_names = train_dt.class_names
print(class_names)

['glioma', 'meningioma', 'notumor', 'pituitary']


In [13]:
AUTOTUNE = tf.data.AUTOTUNE

train_dt = train_dt.cache().prefetch(buffer_size=AUTOTUNE)
val_dt = val_dt.cache().prefetch(buffer_size=AUTOTUNE)

In [14]:
def build_model(input_shape):
    X_input = Input(input_shape)
    X = ZeroPadding2D((2, 2))(X_input)

    X = Conv2D(32, (7, 7), strides=(1, 1))(X)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)

    X = MaxPooling2D((4, 4))(X)
    X = MaxPooling2D((4, 4))(X)
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid')(X)
    model = Model(inputs=X_input, outputs=X)

    return model

In [15]:
IMG_SHAPE = (224, 224, 3)

In [16]:
import pickle

In [17]:
model_1 = tf.keras.Sequential(layers=[
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, activation="relu"),
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=(1, 1)),

    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"),
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=(1, 1)),

    tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation="relu"),
    tf.keras.layers.Conv2D(filters=16, kernel_size=3, activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2, strides=(1, 1)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(32, activation="relu"),

    tf.keras.layers.Dense(4, activation="softmax")
])

model_1.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0014),
                metrics=["accuracy"])

epochs = 100
history_1 = model_1.fit(train_dt,
                        validation_data=val_dt,
                        epochs=epochs,
                        steps_per_epoch=len(train_dt),
                        validation_steps=len(val_dt),
                        verbose=2)


Epoch 1/100
143/143 - 928s - loss: 15.2031 - accuracy: 0.2562 - val_loss: 1.3838 - val_accuracy: 0.2933 - 928s/epoch - 6s/step
Epoch 2/100
143/143 - 885s - loss: 1.3834 - accuracy: 0.2676 - val_loss: 1.3761 - val_accuracy: 0.3047 - 885s/epoch - 6s/step
Epoch 3/100
143/143 - 823s - loss: 1.3905 - accuracy: 0.2650 - val_loss: 1.3665 - val_accuracy: 0.3047 - 823s/epoch - 6s/step
Epoch 4/100
143/143 - 836s - loss: 1.3500 - accuracy: 0.2972 - val_loss: 1.2746 - val_accuracy: 0.3809 - 836s/epoch - 6s/step
Epoch 5/100
143/143 - 1087s - loss: 1.3048 - accuracy: 0.3536 - val_loss: 1.2841 - val_accuracy: 0.3774 - 1087s/epoch - 8s/step
Epoch 6/100
143/143 - 1166s - loss: 1.2940 - accuracy: 0.3626 - val_loss: 1.2506 - val_accuracy: 0.4011 - 1166s/epoch - 8s/step
Epoch 7/100
143/143 - 1174s - loss: 1.2879 - accuracy: 0.3527 - val_loss: 1.2445 - val_accuracy: 0.4011 - 1174s/epoch - 8s/step
Epoch 8/100
143/143 - 958s - loss: 1.2700 - accuracy: 0.3746 - val_loss: 1.2264 - val_accuracy: 0.4203 - 958s/e

Epoch 65/100
143/143 - 1506s - loss: 0.0950 - accuracy: 0.9735 - val_loss: 1.3845 - val_accuracy: 0.7522 - 1506s/epoch - 11s/step
Epoch 66/100
143/143 - 1256s - loss: 0.1080 - accuracy: 0.9726 - val_loss: 1.4359 - val_accuracy: 0.7452 - 1256s/epoch - 9s/step
Epoch 67/100
143/143 - 15935s - loss: 0.1039 - accuracy: 0.9731 - val_loss: 1.2605 - val_accuracy: 0.7653 - 15935s/epoch - 111s/step
Epoch 68/100
143/143 - 2653s - loss: 0.0976 - accuracy: 0.9783 - val_loss: 1.4590 - val_accuracy: 0.7408 - 2653s/epoch - 19s/step
Epoch 69/100
143/143 - 1338s - loss: 0.0818 - accuracy: 0.9779 - val_loss: 1.4156 - val_accuracy: 0.7583 - 1338s/epoch - 9s/step
Epoch 70/100
143/143 - 1371s - loss: 0.1269 - accuracy: 0.9674 - val_loss: 1.4394 - val_accuracy: 0.7601 - 1371s/epoch - 10s/step
Epoch 71/100
143/143 - 5704s - loss: 0.0988 - accuracy: 0.9748 - val_loss: 1.2801 - val_accuracy: 0.7715 - 5704s/epoch - 40s/step
Epoch 72/100
143/143 - 2482s - loss: 0.0896 - accuracy: 0.9759 - val_loss: 1.4107 - val_a

In [18]:
with open("model_1.pkl", "wb") as file:
    pickle.dump(history_1, file)

INFO:tensorflow:Assets written to: ram://db6b1d21-7942-4f5e-aabb-0057543b7417/assets


INFO:tensorflow:Assets written to: ram://db6b1d21-7942-4f5e-aabb-0057543b7417/assets


In [20]:
with open('model_1.pkl','rb') as file:
    pipeline = pickle.load(file)

In [21]:
print(model_1.pkl)

AttributeError: 'Sequential' object has no attribute 'pkl'

In [4]:
import git

ModuleNotFoundError: No module named 'git'